In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

Import csv files Movie details Dataset (CSV 1.52 MB) and Movies online streaming services Dataset (CSV 539 KB).

In [ ]:
movie_details = pd.read_csv('./Dataset-Movie-details.csv')
movies_streaming_services = pd.read_csv('./Dataset-Movies-online-streaming-services.csv')
movies_ranking_join = pd.read_csv('./French_movie_overall_ranking_join.csv')

To get the column names and re-rank the columns by manipulation order.

In [ ]:
movie_details.columns.values.tolist()
order_col = ['Title', 'Language', 'IMDb', 'Rotten Tomatoes', 'Age', 'Genres', 'Country', 'Runtime', 'Year', 'Directors']
movie_details = movie_details[order_col]

List movies that are available in French, then name it to french_movie_only

In [ ]:
movie_details[movie_details['Language'].isnull()]

In [ ]:
Movie_language_null = movie_details[movie_details['Language'].isnull()]
movie_details.drop(index = Movie_language_null.index, inplace = True)

Get the index of 'Language' that contains French 

In [ ]:
French_index = movie_details['Language'].str.contains('French')

In [ ]:
french_movie_only = movie_details[French_index].reset_index().drop(['index'],axis=1)

In [ ]:
french_movie_only.describe(include='all')

Export the french_movie_only to csv file and save in the same folder.

In [ ]:
french_movie_only.to_csv('./French_movie_only.csv')

Find the rowsa that both 'IMDb' and 'Rotten Tomatoes' are nan, then delete these rows.

In [ ]:
french_movie_null_rating_index = (~(french_movie_only['IMDb'].isnull()))|(~(french_movie_only['Rotten Tomatoes'].isnull()))

In [ ]:
french_movie_with_rating = french_movie_only[french_movie_null_rating_index].reset_index().drop(['index'],axis=1)

In [ ]:
french_movie_with_rating.head()

Normalize 'IMDb' and 'Rotten Tomatoes' rating in percentile float data type, then canculate average value.

In [ ]:
french_movie_with_rating.iloc[:, 2] = french_movie_with_rating.iloc[:, 2]*10
french_movie_with_rating.iloc[:, 3] = french_movie_with_rating.iloc[:, 3].apply(lambda x: str(x).split('%')[0] if x != None else x).astype(float)

In [ ]:
french_movie_with_rating

In [ ]:
add_allrating_col_name = french_movie_with_rating.columns.tolist()

Insert a new column 'Overall rating'

In [ ]:
add_allrating_col_name.insert(4,'Overall rating')
french_movie_with_rating = french_movie_with_rating.reindex(columns = add_allrating_col_name)

Calculate mean value of 'IMDb' and 'Rotten Tomatoes'

In [ ]:
french_movie_with_rating['Overall rating'] = french_movie_with_rating[['IMDb', 'Rotten Tomatoes']].mean(axis=1)

In [ ]:
french_movie_overall_ranking = french_movie_with_rating.sort_values('Overall rating', ascending = False, ignore_index = True)

Export the french_movie_overall_ranking to csv file and save in the same folder.

In [ ]:
french_movie_overall_ranking.to_csv('./French_movie_overall_ranking.csv')

Use french_movie_only to find age groups and rank rows according to released year.

In [ ]:
Age_groups = french_movie_only['Age'].unique()
Age_groups

In [ ]:
french_movie_only['Group'] = ''
for i in range(french_movie_only.shape[0]):
    if french_movie_only['Age'][i] == 'all':
        french_movie_only['Group'][i] = 'General'
    elif french_movie_only['Age'][i] == '7+':
        french_movie_only['Group'][i] = 'Parental Guidance'
    elif french_movie_only['Age'][i] == '13+':
        french_movie_only['Group'][i] = 'Mature'
    elif french_movie_only['Age'][i] == '16+' or french_movie_only['Age'][i] == '18+':
        french_movie_only['Group'][i] = 'Mature Accompanied and Restricted'
    else:
        french_movie_only['Group'][i] = 'Unknown rating'

In [ ]:
Add_age_group = ['Title', 'Language', 'IMDb', 'Rotten Tomatoes', 'Age', 'Group', 'Genres', 'Country', 'Runtime', 'Year', 'Directors']
french_movie_only_age_group = french_movie_only[Add_age_group]

Lable age group for ranking purpose

In [ ]:
french_movie_only_age_group['group'] = ''
for i in range(french_movie_only_age_group.shape[0]):
    if french_movie_only_age_group['Age'][i] == 'all':
        french_movie_only_age_group['group'][i] = 0
    elif french_movie_only_age_group['Age'][i] == '7+':
        french_movie_only_age_group['group'][i] = 1
    elif french_movie_only_age_group['Age'][i] == '13+':
        french_movie_only_age_group['group'][i] = 2
    elif french_movie_only_age_group['Age'][i] == '16+' or french_movie_only_age_group['Age'][i] == '18+':
        french_movie_only_age_group['group'][i] = 3
    else:
        french_movie_only_age_group['group'][i] = 4

In [ ]:
french_movie_only_age_group = french_movie_only_age_group.sort_values(['group', 'Year'], ascending=[True, False])

In [ ]:
french_movie_only_age_group.drop(columns = 'group').reset_index()

In [ ]:
french_movie_only_age_group.to_csv('./French_movie_only_age_group.csv')

Find unique genres in french_movie_overall_ranking.

In [ ]:
all_list = []
for i in range(french_movie_overall_ranking.shape[0]):
    all_list += french_movie_overall_ranking['Genres'][i].split(',')

In [ ]:
list_genres = set(all_list)
for i in list_genres:
    french_movie_overall_ranking[i] = ''

In [ ]:
list_genres

In [ ]:
for i in range(french_movie_overall_ranking.shape[0]):
    for k in list_genres:
        if k in french_movie_overall_ranking['Genres'][i]:
            french_movie_overall_ranking[k][i] = 1
        else:
            french_movie_overall_ranking[k][i] = 0

In [ ]:
french_movie_overall_ranking_all_genres = french_movie_overall_ranking

Caculate the total amount at the end of the columns

In [ ]:
sum_genres = french_movie_overall_ranking_all_genres.iloc[:,11:].sum()

In [ ]:
add = {}
for i,j in zip(list_genres,sum_genres):
    add[i] = j

In [ ]:
french_movie_in_genres = french_movie_overall_ranking_all_genres.append(add, ignore_index=True)

In [ ]:
french_movie_in_genres.info()

To get genre rank list

In [ ]:
genres_tp = french_movie_in_genres.transpose()

In [ ]:
genres_tp = genres_tp.sort_values(by = 788, ascending = True)

In [ ]:
genres_tp_tp = genres_tp.transpose()

In [ ]:
genres_tp_tp

In [ ]:
genres_rank_list = genres_tp_tp.columns.values.tolist()

In [ ]:
genres_rank_list

In [ ]:
genres_rank_list_new = ['Title',
 'Language',
 'IMDb',
 'Rotten Tomatoes',
 'Overall rating',
 'News',
 'Short',
 'Sport',
 'Western',
 'Musical',
 'Sci-Fi',
 'Animation',
 'Horror',
 'Music',
 'War',
 'Fantasy',
 'History',
 'Biography',
 'Family',
 'Mystery',
 'Crime',
 'Documentary',
 'Adventure',
 'Action',
 'Thriller',
 'Romance',
 'Comedy',
 'Drama',
 'Age',
 'Genres',
 'Country',
 'Runtime',
 'Year',
 'Directors']

In [ ]:
french_movie_ranking_in_genres = french_movie_in_genres[genres_rank_list_new]

In [ ]:
french_movie_ranking_in_genres.iloc[788, 0] = 'Genre_Count'

In [ ]:
genre_distribution = french_movie_ranking_in_genres.iloc[788, 5:28]

From the bar chart above we can conclude that the 5 most least tapped genres are News, Short, Sport, Western, Musical

In [ ]:
Ranking_pic = genre_distribution.plot.bar(title = 'French movie genre distribution')

In [ ]:
french_movie_ranking_in_each_genres = french_movie_ranking_in_genres.sort_values(['News', 'Short', 'Sport', 'Western', 'Musical', 'Sci-Fi', 'Animation', 'Horror', 'Music', 'War', 'Fantasy', 'History',
 'Biography', 'Family', 'Mystery', 'Crime', 'Documentary', 'Adventure', 'Action', 'Thriller', 'Romance', 'Comedy', 'Drama', 'Overall rating'], ascending =  False).reset_index().drop(['index'],axis=1)

Merge two tables

In [ ]:
french_movie_streaming = pd.merge(movies_ranking_join, movies_streaming_services, on = 'Title')

In [ ]:
french_movie_streaming.head()

In [ ]:
french_movie_streaming = french_movie_streaming.iloc[: , 1:]

Calculate the sum of each service provider and insert the value at bottom

In [ ]:
french_movie_streaming.loc['Total',:]= french_movie_streaming.sum(axis=0)

In [ ]:
french_movie_streaming_provider = french_movie_streaming[["Netflix", "Hulu", "Prime Video", "Disney+"]]

In [ ]:
french_movie_streaming_provider